In [9]:
import Pkg
Pkg.add("Graphs")
Pkg.add("GraphPlot")
Pkg.add("JuMP")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
   Installed DiffRules ─────────── v1.12.1
   Installed DocStringExtensions ─ v0.9.2
   Installed BenchmarkTools ────── v1.3.2
   Installed MutableArithmetics ── v1.0.5
   Installed JuMP ──────────────── v1.4.0
   Installed MathOptInterface ──── v1.9.0
    Updating `~/.julia/environments/v1.8/Project.toml`
  [4076af6c] + JuMP v1.4.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [d360d2e6] + ChainRulesCore v1.15.6
  [9e997f8a] + ChangesOfVariables v0.1.4
  [523fee87] + CodecBzip2 v0.7.2
  [944b1d66] + CodecZlib v0.7.0
  [bbf7d656] + CommonSubexpressions v0.3.0
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + Dif

In [13]:
Pkg.add("CPLEX")

   Resolving package versions...
   Installed CEnum ─ v0.4.2
   Installed CPLEX ─ v0.9.4
    Updating `~/.julia/environments/v1.8/Project.toml`
  [a076750e] + CPLEX v0.9.4
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [fa961155] + CEnum v0.4.2
  [a076750e] + CPLEX v0.9.4
    Building CPLEX → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/32357584cf030fce4b5e2e1852505619283a9d25/build.log`


LoadError: Error building `CPLEX`: 
ERROR: LoadError: Unable to install CPLEX.jl.

The versions of CPLEX supported by CPLEX.jl are:

* 12.10
* 20.1
* 22.1

You must download and install one of these versions separately.

You should set the `CPLEX_STUDIO_BINARIES` environment variable to point to
the install location then try again. For example (updating the path to the
correct location):

```
ENV["CPLEX_STUDIO_BINARIES"] = "/Applications/CPLEX_Studio201/cplex/bin/x86-64_osx/"
import Pkg
Pkg.add("CPLEX")
Pkg.build("CPLEX")
```

See the CPLEX.jl README at https://github.com/jump-dev/CPLEX.jl for further
instructions.

Stacktrace:
 [1] error(s::String)
   @ Base ./error.jl:35
 [2] try_local_installation()
   @ Main ~/.julia/packages/CPLEX/X6Mno/deps/build.jl:139
 [3] top-level scope
   @ ~/.julia/packages/CPLEX/X6Mno/deps/build.jl:157
 [4] include(fname::String)
   @ Base.MainInclude ./client.jl:476
 [5] top-level scope
   @ none:5
in expression starting at /Users/antoine/.julia/packages/

In [10]:
using Graphs
using GraphPlot


###################################################
## Graph: IN/OUT
###################################################

function Read_undirected_Graph_DIMACS(filename)

    g=SimpleGraph(1)	 # Creation of an undirected graph with 1 node

    open(filename) do f
       
           for (i,line) in enumerate(eachline(f))
              x = split(line," ") # For each line of the file, splitted using space as separator
              if(x[1]=="p")       # A line beginning with a 'p' gives the graph size
                  n = parse(Int,x[3])
                  g = SimpleGraph(n)  # Recreation of a undirected graph with n nodes
              elseif(x[1] == "e") # A line beginning with a 'e' gives the edges
                  v_1 = parse(Int, x[2])
                  v_2 = parse(Int, x[3])
                  add_edge!(g,v_1,v_2)
                  #g.weights[v_1,v_2] = 1  # without edge weight
               end
           end
    end
    return g
end


function Read_directed_Graph_GRA(filename)

    g=SimpleDiGraph(1)	 # Creation of a directed graph with 1 node

    open(filename) do f
       
           for (i,line) in enumerate(eachline(f))
              x = split(line," ") # For each line of the file, splitted using space as separator
              if (i==1)
                  n = parse(Int,x[1])  # Read the number of nodes
                  g = SimpleDiGraph(n)  # Recreation of a directed graph with n nodes
              else
                  j=3
                  while (j<size(x)[1]-1)
                    v_1 = parse(Int, x[1])
                    v_2 = parse(Int, x[j])
                    add_edge!(g,v_1+1,v_2+1)   # in this case an edge is oriented as an arc
                    #g.weights[v_1+1,v_2+] = 1  # without arc weight
                    j=j+1
                  end
              end
           end
    end
    return g
end




function WritePdf_visualization_Graph(G,filename)

filename_splitted_in_two_parts = split(filename,".") # split to remove the file extension
filename_with_pdf_as_extension= filename_splitted_in_two_parts[1]*".pdf"
# save to pdf
draw(PDF(filename_with_pdf_as_extension, 16cm, 16cm), gplot(G, nodelabel = 1:nv(G)))
end


WritePdf_visualization_Graph (generic function with 1 method)

In [11]:
using JuMP
using CPLEX
using CPUTime

include("Graph_StableSet.jl")

function PLNE_compact_StableSet(G)

   m = Model(CPLEX.Optimizer)

   @variable(m, x[1:nv(G)], Bin)

   @objective(m, Max, sum(x[i] for i = 1:nv(G) ) )

   for e in edges(G)
       @constraint(m, x[src(e)] + x[dst(e)] <= 1 )
   end

   print(m)
   println()

   optimize!(m)
   
   println(solution_summary(m, verbose=true))

   status = termination_status(m)

   if status == JuMP.MathOptInterface.OPTIMAL
       println("Valeur optimale = ", objective_value(m))
       println("Solution primale optimale :")
      S = Bool[]
       for i= 1:nv(G)
         println("\t x[",i,"] = ", value(x[i]))
         if (value(x[i])<0001) 
           push!(S,0)
         else
           push!(S,1)
         end
       end
       println("Temps de résolution :", solve_time(m))
       return S
   else
      println("Problème lors de la résolution")
   end

end

##########################
### Launch compact PLNE and visualization for the stable set problem
##########################

function Find_exact_StableSet(filename)

 G=Read_undirected_Graph_DIMACS(filename)

 @time @CPUtime S=PLNE_compact_StableSet(G)

 WritePdf_visualisation_Stable(G,S,filename)

end


LoadError: ArgumentError: Package CPLEX not found in current path.
- Run `import Pkg; Pkg.add("CPLEX")` to install the CPLEX package.

In [12]:
Find_exact_StableSet("Instances/DIMACS/0023_myciel4.dim")

LoadError: UndefVarError: Find_exact_StableSet not defined